In [41]:
import requests
import pandas as pd
import defs
from datetime import datetime, timedelta
import pytz
import defs

Debo obtener la data de ESTA semana. Parametros en RFC3339.  
Considerando que solo obtengo 5000 velas por llamada de la API y que cada dia tengo 1440 velasde M1, tengo que hacer 2.016 llamadas de la API por cada toda una semana.  
**NO HAY CONVERSION DE HORAS, LA HORA SOLO ES UNA Y VAMOS A USAR FORMATO Z UTC+0**  
- FORMATO RFC 3339:
    - 2024-07-03T03:08:00.000000000Z  (le resto 4 horas al UTC+0)


Como el API usa los valores Z UTC+0, voy a basar toda mi logica en este formato, bajo esa premisa mis horas importantes son:

- Inicio del dia de NY en formato Z: 04:00:00.000000000Z:
    - 2024-07-03T04:00:00.000000000Z
- Timeframe para Liquidity Grab NY 09:30-10:00 am:
    -   2024-07-03T13:30:00.000000000Z
- Silverbullet Start NY 10:00am:
    -   2024-07-03T14:00:00.000000000Z
- Silverbullet End NY 11:00am :
    -   2024-07-03T15:00:00.000000000Z
- Fin de sesion NY 17:00:
    -   2024-07-03T21:00:00.000000000Z

*NOTA:*  
Notar que en RFC339, la variable Z se usa para indicar que es UTC+0, por lo que la usare siempre (https://medium.easyread.co/understanding-about-rfc-3339-for-datetime-formatting-in-software-engineering-940aa5d5f68a).  


In [180]:
#Funcion para obtener hora actual de NY en formato Z UTC-0
import pytz
from datetime import datetime

def obtener_hora_nueva_york_rfc3339():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Formatear la fecha y hora en el formato deseado
    hora_formateada = ahora_ny.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_formateada

# Ejemplo de uso
hora_actual_ny = obtener_hora_nueva_york_rfc3339()
print("Hora actual en Nueva York:", hora_actual_ny)


Hora actual en Nueva York: 2024-07-03T09:19:21.961195Z


In [187]:
# Validacion de horario de verano en NY (retorno booleano)
import pytz
from datetime import datetime

def esta_en_horario_verano_nueva_york():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Verificar si estamos en horario de verano
    esta_en_horario_verano = ahora_ny.dst() != zona_horaria_ny.dst(None)

    return esta_en_horario_verano

esta_en_horario_verano_nueva_york()

True

In [194]:
# Hora 04:00 del dia de HOY

import pytz
from datetime import datetime, timedelta

def obtener_hora_04_00_utc_rfc3339():
    # Obtener la fecha y hora actual en UTC+0
    ahora_utc = datetime.now(pytz.utc)

    # Crear un nuevo objeto datetime con la hora fijada a las 04:00 del mismo día
    hora_04_00 = ahora_utc.replace(hour=4, minute=0, second=0, microsecond=0)

    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    hora_rfc3339 = hora_04_00.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_rfc3339

# Ejemplo de uso
hora_04_00_utc_rfc3339 = obtener_hora_04_00_utc_rfc3339()
print("Hora 04:00 UTC+0 (RFC3339):", hora_04_00_utc_rfc3339)


Hora 04:00 UTC+0 (RFC3339): 2024-07-03T04:00:00.000000Z


In [203]:
# Obtengo la hora 04:00 de hace 7 dias, validando previamente si es horario de verano 
import pytz
from datetime import datetime, timedelta

def obtener_hora_04_00_utc_rfc3339_hace_7_dias():
    # Obtener la fecha y hora actual en UTC+0
    ahora_utc = datetime.now(pytz.utc)

    # Restar 7 días a la fecha y hora actual
    hace_7_dias = ahora_utc - timedelta(days=7)

    # Validar si es horario de verano
    horario_verano = esta_en_horario_verano_nueva_york()
    
    if horario_verano: 
        # Crear un nuevo objeto datetime con la hora fijada a las 04:00 del mismo día hace 7 días
        hora_ajustada = hace_7_dias.replace(hour=4, minute=0, second=0, microsecond=0)
    else:
        hora_ajustada = hace_7_dias.replace(hour=5, minute=0, second=0, microsecond=0)

    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    hora_rfc3339 = hora_ajustada.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_rfc3339

# Ejemplo de uso
hora_04_00_utc_rfc3339_hace_7_dias = obtener_hora_04_00_utc_rfc3339_hace_7_dias()
print("Hora 04:00 UTC+0 (RFC3339) hace 7 días:", hora_04_00_utc_rfc3339_hace_7_dias)


Hora 04:00 UTC+0 (RFC3339) hace 7 días: 2024-06-26T04:00:00.000000Z


### Llamada al API y conversion a Dataframe

In [128]:
def get_candles (instrument, granularity, starting_date, count):
    session = requests.Session()
    url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"
    params = {
        'granularity' : granularity,
        'price' : "MBA",
        'from' : starting_date,
        'count': count
    }
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    candle_data = response.json()
    return candle_data


def convert_to_df(candle_data):
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]
    our_data = []

    for candle in candle_data["candles"]:
        if candle["complete"] == False:
            continue
        new_dict = {}
        new_dict["time"] = candle["time"]
        new_dict["volume"] = candle["volume"]
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame(our_data)



In [212]:
# PRUEBO LA FUNCION
instrument = "EUR_USD"
granularity = "M1"
starting_date = obtener_hora_04_00_utc_rfc3339_hace_7_dias()

initial_json = get_candles(instrument, granularity, starting_date, 5000)
initial_df = convert_to_df(initial_json)
initial_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-26T04:00:00.000000000Z,20,1.07108,1.07108,1.07097,1.07098,1.07101,1.07101,1.07090,1.07090,1.07115,1.07115,1.07104,1.07106
1,2024-06-26T04:01:00.000000000Z,27,1.07100,1.07100,1.07074,1.07076,1.07092,1.07092,1.07067,1.07069,1.07107,1.07107,1.07082,1.07083
2,2024-06-26T04:02:00.000000000Z,28,1.07074,1.07086,1.07074,1.07086,1.07067,1.07078,1.07067,1.07078,1.07082,1.07093,1.07082,1.07093
3,2024-06-26T04:03:00.000000000Z,23,1.07085,1.07092,1.07085,1.07089,1.07077,1.07084,1.07077,1.07082,1.07093,1.07099,1.07092,1.07096
4,2024-06-26T04:04:00.000000000Z,18,1.07088,1.07094,1.07088,1.07094,1.07081,1.07087,1.07080,1.07086,1.07096,1.07101,1.07095,1.07101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-07-01T16:07:00.000000000Z,45,1.07258,1.07274,1.07258,1.07273,1.07251,1.07267,1.07250,1.07265,1.07265,1.07282,1.07265,1.07281
4996,2024-07-01T16:08:00.000000000Z,29,1.07272,1.07272,1.07266,1.07268,1.07264,1.07264,1.07258,1.07261,1.07279,1.07279,1.07274,1.07275
4997,2024-07-01T16:09:00.000000000Z,32,1.07268,1.07274,1.07266,1.07272,1.07260,1.07267,1.07259,1.07264,1.07275,1.07281,1.07273,1.07279
4998,2024-07-01T16:10:00.000000000Z,60,1.07272,1.07278,1.07264,1.07264,1.07265,1.07271,1.07257,1.07257,1.07279,1.07285,1.07272,1.07272


### Logica de Iteracion para obtencion de datos consecutiva del API

In [217]:
#Funcion para obtener la ultima vela del df anterior

def last_candle(candles_df):
    valor = candles_df.iloc[-1, 0]
    return valor

last_candle(initial_df)

'2024-07-01T16:11:00.000000000Z'

In [161]:
# Intento hacer la llamada al API dentro de otra funcion que llame desde una semana atras 

starting_date = obtener_hora_04_00_utc_rfc3339_hace_7_dias()
instrument = "EUR_USD"
granularity = "M1"

def get_historical_candles(starting_date, instrument, granularity, count=5000):
    today_date = obtener_hora_nueva_york_rfc3339()
    json_data = get_candles(instrument, granularity, starting_date, 5000)
    df = convert_to_df(json_data)
    last_previous_candle = last_candle(df)
    # while last_previous_candle!= today_date:
    #     starting_date = last_previous_candle['time']
    #     json_data = get_candles(instrument, granularity, starting_date, 5000)
    #     df = df.append(convert_to_df(json_data))
    #     last_previous_candle = last_candle(df)
    last_candle = df.iloc[-1, 0]
    if 
    

    return df

get_historical_candles(starting_date, instrument, granularity)
    

<class 'str'>


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-25T04:01:00.000000000Z,37,1.07383,1.07387,1.07383,1.07386,1.07376,1.07380,1.07376,1.07379,1.07390,1.07394,1.07390,1.07394
1,2024-06-25T04:02:00.000000000Z,17,1.07387,1.07387,1.07383,1.07384,1.07380,1.07380,1.07375,1.07376,1.07394,1.07395,1.07391,1.07391
2,2024-06-25T04:03:00.000000000Z,4,1.07382,1.07383,1.07380,1.07380,1.07375,1.07376,1.07373,1.07373,1.07389,1.07390,1.07388,1.07388
3,2024-06-25T04:04:00.000000000Z,15,1.07380,1.07381,1.07380,1.07380,1.07372,1.07374,1.07372,1.07372,1.07388,1.07388,1.07387,1.07387
4,2024-06-25T04:05:00.000000000Z,24,1.07380,1.07380,1.07372,1.07376,1.07373,1.07373,1.07365,1.07368,1.07387,1.07387,1.07380,1.07383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-06-28T16:19:00.000000000Z,41,1.07162,1.07169,1.07162,1.07167,1.07155,1.07162,1.07155,1.07160,1.07170,1.07176,1.07170,1.07174
4996,2024-06-28T16:20:00.000000000Z,49,1.07168,1.07173,1.07165,1.07172,1.07160,1.07166,1.07158,1.07165,1.07175,1.07180,1.07172,1.07180
4997,2024-06-28T16:21:00.000000000Z,31,1.07173,1.07186,1.07166,1.07186,1.07166,1.07179,1.07159,1.07179,1.07180,1.07194,1.07174,1.07194
4998,2024-06-28T16:22:00.000000000Z,53,1.07187,1.07196,1.07182,1.07187,1.07180,1.07189,1.07175,1.07180,1.07194,1.07204,1.07189,1.07194


In [162]:
obtener_hora_nueva_york_rfc3339()

'2024-07-02T10:18:26.779180Z'

# UTILIDADES A NECESITAR LUEGO

In [40]:
#Como eliminar el duplicado, hacer luego de juntar los dataframes
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'Nombre': ['Ana', 'Luis', 'Ana', 'Carlos', 'Luis'],
        'Edad': [23, 34, 23, 45, 34],
        'Ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Barcelona']}

df = pd.DataFrame(data)

# Mostrar el DataFrame original
print("DataFrame original:")
print(df)

# Eliminar filas duplicadas
df_sin_duplicados = df.drop_duplicates()

# Mostrar el DataFrame sin duplicados
print("\nDataFrame sin duplicados:")
print(df_sin_duplicados)

DataFrame original:
   Nombre  Edad     Ciudad
0     Ana    23     Madrid
1    Luis    34  Barcelona
2     Ana    23     Madrid
3  Carlos    45   Valencia
4    Luis    34  Barcelona

DataFrame sin duplicados:
   Nombre  Edad     Ciudad
0     Ana    23     Madrid
1    Luis    34  Barcelona
3  Carlos    45   Valencia


In [179]:
# Este es el que vale
import pytz
from datetime import datetime

def obtener_hora_actual_nueva_york_rfc3339():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Convertir la fecha y hora a UTC+0
    ahora_utc = ahora_ny.astimezone(pytz.utc)

    # Formatear la fecha y hora en formato RFC3339 con la 'Z'
    hora_rfc3339 = ahora_utc.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_rfc3339

# Ejemplo de uso
hora_actual_ny_rfc3339 = obtener_hora_actual_nueva_york_rfc3339()
print("Hora actual en Nueva York (RFC3339 UTC+0):", hora_actual_ny_rfc3339)


Hora actual en Nueva York (RFC3339 UTC+0): 2024-07-03T04:46:38.492565Z


In [183]:
# Validacion de horario de verano
import pytz
from datetime import datetime

def esta_en_horario_verano_nueva_york():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Verificar si estamos en horario de verano
    esta_en_horario_verano = ahora_ny.dst() != zona_horaria_ny.dst(None)

    return esta_en_horario_verano

# Ejemplo de uso
if esta_en_horario_verano_nueva_york():
    print("Estamos en horario de verano en Nueva York o sea UTC-4")
else:
    print("No estamos en horario de verano en Nueva York o sea UTC-5")



Estamos en horario de verano en Nueva York o sea UTC-4


True